In [2]:
import cv2
from threading import Thread, Event
from queue import Queue
import time

In [3]:
class Holder:
    queue_dynamic = Queue()
    queue_gesture = Queue()
    event = Event()
    
    def push(img):
        Holder.queue_dynamic.put(img)
        Holder.queue_gesture.put(img)
        
    def get_dynamic():
        if Holder.queue_dynamic.empty():
            return None
        return Holder.queue_dynamic.get()
    
    def get_gesture():
        if Holder.queue_gesture.empty():
            return None
        return Holder.queue_gesture.get()

class CameraThread(Thread):        
    def run(self):
        self.cap = cv2.VideoCapture(0)
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print('capture read None!')
                break
            Holder.push(frame)
            if Holder.event.isSet():
                break
            cv2.imshow('camera-threading', frame)
            if cv2.waitKey(10) == ord('q'):  # 点击视频，输入q退出
                self.cap.release()
                break
        cv2.destroyAllWindows()

In [5]:
class DynamicPose(Thread):
    def run(self):
        while True:
            frame = Holder.get_dynamic()
#             frame = cv2.flip(frame, 1)
#             cv2.putText(frame, 'dynamic', (200, 100), 
#                         cv2.FONT_HERSHEY_COMPLEX, 2.0, (100, 200, 200))
            cv2.imshow('dynamic', frame)
            if cv2.waitKey(10) == ord('q'):  # 点击视频，输入q退出
                break
        cv2.destroyAllWindows()

In [6]:
ct = CameraThread()
dp = DynamicPose()

In [7]:
ct.start()
dp.start()
# ct.join()
# dp.join()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "D:\program_pro\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-5-e11bd581480b>", line 9, in run
cv2.error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-k1ohfcms\opencv\modules\highgui\src\window.cpp:404: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'

